In [1]:
import pandas as pd 
import numpy as np
import os 

In [2]:
pwd = os.getcwd()

In [3]:
import networkx as nx

all_graphs = {}

# 将节点之间方向相同的边进行合并，权值为边的数量与边的值和
node_graphs = {}

node_features = pd.DataFrame()

for filename in os.listdir(pwd + '/钓鱼一阶节点'):
    if filename.endswith('.csv'):
        data = pd.read_csv(pwd + '/钓鱼一阶节点/' + filename)
        # 构建一阶交易图，两个节点之间存在多条边
        G = nx.MultiDiGraph()
        for i in range(len(data)):
            G.add_edge(data['From'][i], data['To'][i], value = data['Value'][i], timestamp = data['TimeStamp'][i])
        
        # 将节点之间方向相同的边进行合并，权值为边的数量与边的值和
        G1 = nx.DiGraph()
        # 将data按照From和To分组，然后对每个分组进行遍历，将value相加
        for name, group in data.groupby(['From', 'To']):
            G1.add_edge(name[0], name[1], value = group['Value'].sum(), timestamp = group['TimeStamp'].max(), count = len(group))
        
        # 计算节点特征
        features = {}
        features['address'] = filename.split('.')[0]
        features['in_degree'] = len(G.in_degree())
        features['out_degree'] = len(G.out_degree())
        features['max_timestamp'] = data['TimeStamp'].max()
        features['min_timestamp'] = data['TimeStamp'].min()
        features['value_out'] = data[data['From'] == filename.split('.')[0]]['Value'].sum()
        features['value_in'] = data[data['To'] == filename.split('.')[0]]['Value'].sum()
        features['balance'] = features['value_out'] - features['value_in']
        features['count'] = len(data)
        features['label'] = 0

        # 将节点特征添加到node_features中
        node_features = node_features.append(features, ignore_index = True)

        all_graphs[filename.split('.')[0]] = G
        node_graphs[filename.split('.')[0]] = G1

C:\Users\YangXikang\AppData\Local\Temp\ipykernel_27808\4196666915.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  node_features = node_features.append(features, ignore_index = True)
C:\Users\YangXikang\AppData\Local\Temp\ipykernel_27808\4196666915.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  node_features = node_features.append(features, ignore_index = True)
C:\Users\YangXikang\AppData\Local\Temp\ipykernel_27808\4196666915.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  node_features = node_features.append(features, ignore_index = True)
C:\Users\YangXikang\AppData\Local\Temp\ipykernel_27808\4196666915.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

In [4]:
for filename in os.listdir(pwd + '/非钓鱼一阶节点'):
    if filename.endswith('.csv'):
        data = pd.read_csv(pwd + '/非钓鱼一阶节点/' + filename)
        # 构建一阶交易图，两个节点之间存在多条边
        G = nx.MultiDiGraph()
        for i in range(len(data)):
            G.add_edge(data['From'][i], data['To'][i], value = data['Value'][i], timestamp = data['TimeStamp'][i])
        
        # 将节点之间方向相同的边进行合并，权值为边的数量与边的值和
        G1 = nx.DiGraph()
        # 将data按照From和To分组，然后对每个分组进行遍历，将value相加
        for name, group in data.groupby(['From', 'To']):
            G1.add_edge(name[0], name[1], value = group['Value'].sum(), timestamp = group['TimeStamp'].max(), count = len(group))

        
        # 计算节点特征
        features = {}
        features['address'] = filename.split('.')[0]
        features['in_degree'] = len(G.in_degree())
        features['out_degree'] = len(G.out_degree())
        features['max_timestamp'] = data['TimeStamp'].max()
        features['min_timestamp'] = data['TimeStamp'].min()
        features['value_out'] = data[data['From'] == filename.split('.')[0]]['Value'].sum()
        features['value_in'] = data[data['To'] == filename.split('.')[0]]['Value'].sum()
        features['balance'] = features['value_out'] - features['value_in']
        features['count'] = len(data)
        features['label'] = 1

        # 将节点特征添加到node_features中
        node_features = node_features.append(features, ignore_index = True)

        all_graphs[filename.split('.')[0]] = G
        node_graphs[filename.split('.')[0]] = G1

C:\Users\YangXikang\AppData\Local\Temp\ipykernel_27808\2750678660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  node_features = node_features.append(features, ignore_index = True)
C:\Users\YangXikang\AppData\Local\Temp\ipykernel_27808\2750678660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  node_features = node_features.append(features, ignore_index = True)
C:\Users\YangXikang\AppData\Local\Temp\ipykernel_27808\2750678660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  node_features = node_features.append(features, ignore_index = True)
C:\Users\YangXikang\AppData\Local\Temp\ipykernel_27808\2750678660.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

In [5]:
# 二阶交易图
for address in os.listdir(pwd + '/钓鱼二阶节点'):
    if address == '.DS_Store':
        continue
    print("Processing address: ", address)

    # 最多取前500个节点
    count = 0

    for filename in os.listdir(pwd + '/钓鱼二阶节点/' + address):
        G = all_graphs[address].copy()
        G1 = node_graphs[address].copy()

        if filename.endswith('.csv'):
            if count == 500:
                break
            count += 1
            
            data = pd.read_csv(pwd + '/钓鱼二阶节点/' + address + '/' + filename)
            # 构建一阶交易图，两个节点之间存在多条边

            for i in range(len(data)):
                G.add_edge(data['From'][i], data['To'][i], value = data['Value'][i], timestamp = data['TimeStamp'][i])
            
            for name, group in data.groupby(['From', 'To']):
                G1.add_edge(name[0], name[1], value = group['Value'].sum(), timestamp = group['TimeStamp'].max(), count = len(group))

            all_graphs[filename.split('.')[0]] = G
            node_graphs[filename.split('.')[0]] = G1

Processing address:  0x002bf459dc58584d58886169ea0e80f3ca95ffaf
Processing address:  0x002f0c8119c16d310342d869ca8bf6ace34d9c39
Processing address:  0x0059b14e35dab1b4eee1e2926c7a5660da66f747
Processing address:  0x0061fb5485dff4bb85c078dca80d19119224d97e
Processing address:  0x00650a784ee109797ba8c05f3496bdbf2b2b7a1c
Processing address:  0x0084515449b037205a33d6d3940a5684126aa4b5
Processing address:  0x00c33c49f9a2a920e3f3787204cbda9012d1912e
Processing address:  0x00eb6f5199cd0b671da371969b1a0f948e982fea
Processing address:  0x0128282ce73c72decabaeace9358344adff449fe
Processing address:  0x015c0e438b3a01511b98d928bd031d3dc50abb9e
Processing address:  0x015fca1d09287823e634ae51237ee1eeff03d9d5
Processing address:  0x0167409e6106ec3e3f05a09fcf04606918d21ad5
Processing address:  0x0177eb92b752fa0715ee0dce1d860eaf739b5cf4
Processing address:  0x017f86b90a46d8fd999eaefda1339355b98da12f
Processing address:  0x018f2f4b3c9e38607aa7ab4dea23e9d663e3e050
Processing address:  0x020b1573f2ca67019

In [6]:
# 二阶交易图
for address in os.listdir(pwd + '/非钓鱼二阶节点'):
    if address == '.DS_Store':
        continue
    print("Processing address: ", address)

    count = 0 # 最多取前500个节点

    for filename in os.listdir(pwd + '/非钓鱼二阶节点/' + address):
        G = all_graphs[address].copy()
        G1 = node_graphs[address].copy()

        if filename.endswith('.csv'):
            if count == 500:
                break
            count += 1

            data = pd.read_csv(pwd + '/非钓鱼二阶节点/' + address + '/' + filename)
            # 构建一阶交易图，两个节点之间存在多条边

            for i in range(len(data)):
                G.add_edge(data['From'][i], data['To'][i], value = data['Value'][i], timestamp = data['TimeStamp'][i])
            
            for name, group in data.groupby(['From', 'To']):
                G1.add_edge(name[0], name[1], value = group['Value'].sum(), timestamp = group['TimeStamp'].max(), count = len(group))

            all_graphs[filename.split('.')[0]] = G
            node_graphs[filename.split('.')[0]] = G1

Processing address:  0x0000000000000000000000000000000000000000
Processing address:  0x000419c40a811a052b56372f4a80823be47db756
Processing address:  0x000e0e5701b14fb77160bcc7bfe7256522d5927b
Processing address:  0x00267e4a01f25072e0e9347a2ccdd67091d7aeac
Processing address:  0x00316d956f5f35591ae021f4858a2a865c6ba02a
Processing address:  0x0034cf6e02f4c47fb30df22fc81b8dedddbf1fb0
Processing address:  0x004e3def0c754a921af751d1004df95f9650ea00
Processing address:  0x005bdf2845064db405f5c99aaf9510d0b19e7ac2
Processing address:  0x005e288d713a5fb3d7c9cf1b43810a98688c7223
Processing address:  0x006deef69f1b3c01173b5ba74a5de9050e72d702
Processing address:  0x007c508c6368d2ad35608cb8e98edc9ef1bf0e84
Processing address:  0x00a2df284ba5f6428a39dff082ba7ff281852e06
Processing address:  0x00a80d1f0649358af50c5b9263777e4b1dcec366
Processing address:  0x00ce8ad7e44fea30ecbe2fb69963eaa26b5b56a0
Processing address:  0x00d681a7b6584f978f63c81cfd847064ce19a080
Processing address:  0x00e5c013694c9ee92

In [ ]:
test_G = all_graphs['0xfe68f28599b19c5d8a562e8cc7f7b07c36e0a99d']

import random
import matplotlib.pyplot as plt

# 画出图的结构
def draw_graph(G):
    plt.figure(figsize=(10,10))
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, node_size = 10)
    nx.draw_networkx_edges(G, pos, alpha = 0.3)
    plt.show()

draw_graph(test_G)

In [ ]:
# 带重启的随机游走，有概率返回到起点，有概率继续游走, 当游走到终点时，返回到起点, 可以反向游走
def random_walk(G, start, walk_length, restart_prob):
    walk = [start]
    while len(walk) < walk_length:
        cur = walk[-1]
        if random.random() < restart_prob:
            walk.append(random.choice(list(G.nodes())))
        else:
            if len(list(G.neighbors(cur))) == 0:
                walk.append(random.choice(list(G.nodes())))
            else:
                walk.append(random.choice(list(G.neighbors(cur))))

    # 获取游走子图
    subgraph = G.subgraph(walk)
    return subgraph

# 生成游走子图
def generate_walks(G, num_walks, walk_length, restart_prob):
    walks = []
    nodes = list(G.nodes())

    while len(walks) < num_walks:
        random.shuffle(nodes)
        random_start_node = random.choice(nodes)
        graph = random_walk(G, random_start_node, walk_length, restart_prob)
        # 是否为存在孤立节点的子图
        if len(list(nx.isolates(graph))) == 0:
            walks.append(graph)
            
    return walks


In [ ]:
# 生成游走子图
print(G.nodes())

walks = generate_walks(test_G, 10, 20, 0.9)

# 画出游走子图, 画在一张图上, 每行5个子图
fig, axs = plt.subplots(2, 5, figsize=(20, 10))
for walk in walks:
    pos = nx.spring_layout(walk)
    nx.draw_networkx_nodes(walk, pos, node_size = 10, ax = axs[walks.index(walk) // 5][walks.index(walk) % 5])
    nx.draw_networkx_edges(walk, pos, alpha = 0.3, ax = axs[walks.index(walk) // 5][walks.index(walk) % 5])
plt.show()


In [ ]:
motifs = {
    'motif1': nx.DiGraph([(0, 1), (1, 2)]),
    'motif2': nx.DiGraph([(0, 1), (2, 1)]),
    'motif3': nx.DiGraph([(0, 1), (0, 2)]),
    'motif4': nx.DiGraph([(0, 1), (1, 2), (2, 0)]),
}

# 提取motifs特征
motif_features = {}

# 从node_graphs中提取motifs特征, 统计motifs的数量

for key in node_graphs.keys():
    print("Process: {}".format(key))
    # 提取motifs特征
    motif_features[key] = {}
    for motif in motifs.keys():
        motif_features[key][motif] = 0
    
    # 随机游走获取子图
    walks = generate_walks(node_graphs[key], 5, 15, 0.9)

    sub_graph = node_graphs[key]

    for walk in walks:
        # 找出sub_graph中所有匹配motifs的结构， 并统计数量
        for motif in motifs:
            for subgraph in nx.algorithms.isomorphism.GraphMatcher(walk, motifs[motif]).subgraph_isomorphisms_iter():
                motif_features[key][motif] += 1

# 将motif_features转化为df
motif_features = pd.DataFrame(motif_features)
motif_features = motif_features.T

motif_features

In [ ]:
# 画motifs, 每行5
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
for i, key in enumerate(motifs.keys()):
    plt.subplot(1, 5, i + 1)
    nx.draw(motifs[key], with_labels=False, node_size=30, width=1)
    plt.title(key)

In [ ]:
# 将index转化为列， 并将index重命名为address
motif_features = motif_features.reset_index()
motif_features = motif_features.rename(columns={'index': 'address'})

motif_features.describe()

In [ ]:
# 计算每种motif占所有motif的比例
motif_features['motif1_ratio'] = motif_features['motif1'] / (motif_features['motif1'] + motif_features['motif2'] + motif_features['motif3'] + motif_features['motif4'])
motif_features['motif2_ratio'] = motif_features['motif2'] / (motif_features['motif1'] + motif_features['motif2'] + motif_features['motif3'] + motif_features['motif4'])
motif_features['motif3_ratio'] = motif_features['motif3'] / (motif_features['motif1'] + motif_features['motif2'] + motif_features['motif3'] + motif_features['motif4'])
motif_features['motif4_ratio'] = motif_features['motif4'] / (motif_features['motif1'] + motif_features['motif2'] + motif_features['motif3'] + motif_features['motif4'])

In [ ]:
motif_features.fillna(0, inplace=True)

In [ ]:
# 将node_features和motif_features合并
featuresX = pd.merge(node_features, motif_features, on='address', how='left')

In [ ]:
# 数据划分
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# xgb
import xgboost as xgb
# 随机森林
from sklearn.ensemble import RandomForestClassifier
# svm
from sklearn import svm
# 神经网络
from sklearn.neural_network import MLPClassifier
# 逻辑回归
from sklearn.linear_model import LogisticRegression
# 朴素贝叶斯
from sklearn.naive_bayes import GaussianNB
# 隔离森林
from sklearn.ensemble import IsolationForest
# KNN
from sklearn.neighbors import KNeighborsClassifier
# 决策树
from sklearn.tree import DecisionTreeClassifier
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
# GBDT
from sklearn.ensemble import GradientBoostingClassifier
# lightGBM
import lightgbm as lgb


In [ ]:
# node_features_columns = ['in_degree', 'out_degree', 'max_timestamp', 'min_timestamp', 'value_out', 'value_in', 'balance', 'count']
node_features_columns = ['balance', 'count']


# 使用node_features_columns中的特征
trainX, testX, trainY, testY = train_test_split(featuresX[node_features_columns], featuresX['label'], test_size=0.3, random_state=0)

# 训练模型
clf = lgb.LGBMClassifier()
clf.fit(trainX, trainY)

# 预测
pred = clf.predict(testX)

# 评估
print(classification_report(testY, pred))

In [ ]:
motif_features_columns = ['motif1_ratio', 'motif2_ratio', 'motif3_ratio']

# 使用motif_features_columns中的特征
trainX, testX, trainY, testY = train_test_split(featuresX[motif_features_columns], featuresX['label'], test_size=0.3, random_state=0)

# 训练模型
clf = lgb.LGBMClassifier()
clf.fit(trainX, trainY)

# 预测
pred = clf.predict(testX)

# 评估
print(classification_report(testY, pred))


In [ ]:
# 使用node_features_columns和motif_features_columns中的特征
merged_columns = node_features_columns + motif_features_columns
print(merged_columns)

trainX, testX, trainY, testY = train_test_split(featuresX[merged_columns], featuresX['label'], test_size=0.3, random_state=0)

# 训练模型
clf =  lgb.LGBMClassifier()
clf.fit(trainX, trainY)

# 预测
pred = clf.predict(testX)

# 评估
print(classification_report(testY, pred))